In [1]:
# Loading and Preprocessing Images

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Function to load and preprocess the images

def load_images(directory, image_size):
    images = []
    labels = []
    for filename in os.listdir(directory):
        image_path = os.path.join(directory, filename)
        image = cv2.imread(image_path)
        image = cv2.resize(image, image_size)  # Resize images to a uniform size
        image = image.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
        images.append(image)
        # Assuming filename format is "class_filename.jpg"
        class_name = filename.split('_')[0]
        labels.append(class_name)
    return np.array(images), np.array(labels)

# Define directory
data_directory = r"C:\Users\Dell\Documents\Dataset"

# Define image size for resizing
image_size = (128, 128)

# Load and preprocess images

images, labels = load_images(data_directory, image_size)

# Split the dataset into training, validation, and test sets

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Print shapes of the datasets

print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)


Training set shape: (144, 128, 128, 3)
Validation set shape: (48, 128, 128, 3)
Test set shape: (48, 128, 128, 3)


In [2]:
# Annotation of Images using Bounding Boxes

import cv2
import os
import json

# Function to annotate images with bounding boxes

def annotate_with_bounding_boxes(image_paths, annotations_file, batch_size=10):
    annotations = []
    num_images = len(image_paths)

    # Creating a named window for displaying images
    cv2.namedWindow("Annotate with Bounding Boxes")

    for i in range(0, num_images, batch_size):
        batch_image_paths = image_paths[i:i+batch_size]

        for image_path in batch_image_paths:
            # Loading the Image
            image = cv2.imread(image_path)
            if image is None:
                continue

            # Displaying the image and annotating with bounding box

            bounding_box = cv2.selectROI("Annotate with Bounding Boxes", image, fromCenter=False, showCrosshair=True)
            cv2.destroyAllWindows()

            x, y, w, h = bounding_box
            x_min, y_min, x_max, y_max = x, y, x + w, y + h

            # Creating an annotation dictionary

            annotation = {
                "image_path": image_path,
                "bounding_box": [int(x_min), int(y_min), int(x_max), int(y_max)]
            }
            annotations.append(annotation)

    # Saving all annotations to a JSON file
    with open(annotations_file, "w") as f:
        json.dump(annotations, f, indent=4)

    # Destroy or Killing the named window created above
    if cv2.getWindowProperty("Annotate with Bounding Boxes", cv2.WND_PROP_VISIBLE) >= 1:
        cv2.destroyWindow("Annotate with Bounding Boxes")

# Defining the directory containing the images
data_directory = r"C:\Users\Dell\Documents\Dataset"

# Getting paths of all image files in the directory
image_paths = [os.path.join(data_directory, filename) for filename in os.listdir(data_directory) if filename.endswith(".jpg")]

# Setting path for annotations file
annotations_file = "flower_annotations.json"

# Annotating images with bounding boxes and save annotations to file
annotate_with_bounding_boxes(image_paths, annotations_file)

In [9]:
# Training the Model

import os
import cv2
import numpy as np
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Function to load and preprocess the images

def load_images(directory, image_size):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))
    num_classes = len(class_names)
    
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            image_path = os.path.join(class_dir, filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, image_size)  # Resizing images to a uniform size
            image = image.astype(np.float32) / 255.0  # Normalizing pixel values to [0, 1]
            images.append(image)
            labels.append(class_names.index(class_name))
    
    return np.array(images), np.array(labels), num_classes

# Directory containing the images
data_directory = r"C:\Users\Dell\Documents\Dataset"

# Defining image size for resizing
image_size = (128, 128)  # Adjust as needed

# Load and preprocess images
images, labels, num_classes = load_images(data_directory, image_size)

# Split the dataset into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Define CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Saving the model
model.save("flower_classification_model.h5")

# Saving the model history
with open("model_history.json", "w") as f:
    json.dump(history.history, f)




Epoch 1/10


5/5 [==============================] - 3s 332ms/step - loss: 1.1709 - accuracy: 0.4236 - val_loss: 1.0941 - val_accuracy: 0.3125
Epoch 2/10
5/5 [==============================] - 1s 251ms/step - loss: 0.9781 - accuracy: 0.4583 - val_loss: 0.8710 - val_accuracy: 0.5208
Epoch 3/10
5/5 [==============================] - 1s 252ms/step - loss: 0.8070 - accuracy: 0.6319 - val_loss: 0.7067 - val_accuracy: 0.6667
Epoch 4/10
5/5 [==============================] - 1s 259ms/step - loss: 0.6336 - accuracy: 0.7431 - val_loss: 0.5882 - val_accuracy: 0.7083
Epoch 5/10
5/5 [==============================] - 1s 265ms/step - loss: 0.5297 - accuracy: 0.7986 - val_loss: 0.4909 - val_accuracy: 0.7917
Epoch 6/10
5/5 [==============================] - 1s 258ms/step - loss: 0.4197 - accuracy: 0.8403 - val_loss: 0.4912 - val_accuracy: 0.7708
Epoch 7/10
5/5 [==============================] - 1s 255ms/step - loss: 0.3212 - accuracy: 0.8681 - val_loss: 0.6977 - val_accuracy: 0.6250
Epoch 8/10
5/5 

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
# Testing the Model on Data from different Directory

import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Loading the trained model which is created above
model = load_model("flower_classification_model.h5")

# Load and preprocess the input image
def preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image.astype(np.float32) / 255.0  # Normalize pixel values
    return np.expand_dims(image, axis=0)  # Add batch dimension

# Defining the target image size
target_size = (128, 128)

# Path to the input image
input_image_path = r"C:/Users/Dell/Downloads/image_0001.jpg"

# Preprocessing the input image
input_image = preprocess_image(input_image_path, target_size)

# Performing inference to get class probabilities
class_probabilities = model.predict(input_image)

# Predicted class index (class with highest probability)
predicted_class_index = np.argmax(class_probabilities)

# Class labels
class_labels = ['Class1', 'Class2', 'Class3']

# Output
predicted_class_label = class_labels[predicted_class_index]
print("Predicted Class:", predicted_class_label)




1/1 [==============================] - 0s 441ms/step
Predicted Class: Class1
